# Activation Maximization

Implement the unit-activation-maximization algorithm described in https://arxiv.org/pdf/1312.6034.pdf.

In [ ]:
UNIT_NAMES = [
  'sulphur-crested_cockatoo',
  'white_wolf',
  'bulbul',
  'magpie',
  'obelisk',
  'goldfish',
  'brown_bear',
  'assault_rifle',
  'golden_retriever',
  'airliner',
  'dalmatian',
  'chow',
  'tabby',
  'garter_snake',
]

LR = 10.
L2 = .01
TV = .01
STEPS = 300

## Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

! cp -r '/content/drive/MyDrive/Colab Notebooks/cs-no/6.grads/config' .
! pip install -q tensorflow-addons

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
from config import config, data, model, utils

utils.setup_clean_image_plotting()

In [ ]:
from tensorflow.python.keras.applications import mobilenet_v2

config.model.architecture = tf.keras.applications.MobileNetV2
config.model.module = mobilenet_v2
config.model.decode = tf.keras.applications.mobilenet_v2.decode_predictions
config.model.preprocess = tf.keras.applications.mobilenet_v2.preprocess_input
config.model.decompress = lambda x: (x + 1.) * 127.5

config.data.image_size = (224, 224)
config.data.input_shape = (config.data.batch_size, 224, 224, 3)

In [ ]:
images = data.load_images()
x = model.preprocess(images)

In [ ]:
nn = model.build()

logits = nn(x, training=False)
preds = tf.argmax(logits, axis=1)
probs = tf.nn.softmax(logits)
decoded = model.decode(probs.numpy(), top=2)

## Original Images and Classification Results

In [ ]:
utils.plot(utils.to_image(images),
           titles=[f'{f}\n{p[0][1]} {p[0][2]:.0%}'
                  for f, p in zip(data.class_names, decoded)],
           rows=4)

Output hidden; open in https://colab.research.google.com to view.

## Observing Class-Model Visualization

In [ ]:
def activation_loss(y, unit):
    return tf.reduce_mean(y[:, unit])

def l2_regularization(x):
    return tf.reduce_sum(tf.square(x), axis=(1, 2, 3), keepdims=True)

def total_var_regularization(x):
    return tf.reduce_sum(tf.image.total_variation(x) / tf.cast(tf.reduce_prod(x.shape[1:-1]), tf.float32))

@tf.function
def gradient_ascent_step(inputs, unit):
    with tf.GradientTape() as tape:
        tape.watch(inputs)
        y = nn(x)

        loss = (activation_loss(y, unit)
                - L2 * l2_regularization(inputs)
                - TV * total_var_regularization(inputs))

    grads = tape.gradient(loss, inputs)
    grads = tf.math.l2_normalize(grads)
    inputs += LR * grads
    return loss, inputs

In [ ]:
import numpy as np

def _gaussian_kernel(kernel_size, sigma, n_channels, dtype):
    # https://stackoverflow.com/questions/59286171/gaussian-blur-image-in-dataset-pipeline-in-tensorflow
    x = tf.range(-kernel_size // 2 + 1, kernel_size // 2 + 1, dtype=dtype)
    g = tf.math.exp(-(tf.pow(x, 2) / (2 * tf.pow(tf.cast(sigma, dtype), 2))))
    g_norm2d = tf.pow(tf.reduce_sum(g), 2)
    g_kernel = tf.tensordot(g, g, axes=0) / g_norm2d
    g_kernel = tf.expand_dims(g_kernel, axis=-1)
    return tf.expand_dims(tf.tile(g_kernel, (1, 1, n_channels)), axis=-1)

BLUR_K = _gaussian_kernel(3, 2, 3, tf.float32)


def apply_blur(img):
    img = tf.nn.depthwise_conv2d(img, BLUR_K, [1,1,1,1], 'SAME')
    return img

def augment(i):
    i = tfa.image.rotate(i, np.random.randn() * 0.05)
    i = tf.roll(i, (np.random.randn(2) * 3).astype(int), (1, 2))
    # i = apply_blur(i)
    return i

In [ ]:
def visualize_filter(unit, steps=STEPS):
    i = tf.random.uniform((1, *config.data.image_size, 3))
    i = (i - 0.5) * 0.25

    for step in range(steps):
        i = augment(i)

        loss, i = gradient_ascent_step(i, unit)
        if step < STEPS - 1:
            i = tf.clip_by_value(i, -1, 1)

    return loss, i

In [ ]:
indices = [utils.unit_index_from(u) for u in UNIT_NAMES]
o = tf.concat([visualize_filter(u)[1] for u in indices], axis=0)

logits = nn(o, training=False)
preds = tf.argmax(logits, axis=1)
probs = tf.nn.softmax(logits)
decoded = model.decode(probs.numpy(), top=2)

In [ ]:
utils.plot(utils.to_image(model.decompress(utils.normalize(o))), # utils.to_image(model.decompress(o)),
           titles=[f'maximizing: {u}\npredicted: {p[0][1]} {p[0][2]:.0%}'
                   for u, p in zip(UNIT_NAMES, decoded)],
           rows=4)

Output hidden; open in https://colab.research.google.com to view.